In [ ]:
# %% Pirate Weather Example script for SMSL  

import os
import json
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os
import shutil
import sys
import uuid
from urllib.parse import unquote_plus

from netCDF4 import Dataset
import numpy as np

# Plotting tools
import matplotlib.pyplot as plt

# Date processing
from datetime import datetime, timedelta

#os.system('apt install gfortran')

# Wgrib2 setup
import pywgrib2_s

In [ ]:
# S3 Client Setup
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3 = boto3.resource('s3')


In [ ]:
#%% Set download variables
bucket          = 'noaa-hrrr-bdp-pds'
download_path   = '/tmp'
time_in         = '2022-09-16T00:00:00'
product         = 'hrrr'

# Datetime Setup
datetime_RUN        = datetime.strptime(time_in,"%Y-%m-%dT%H:%M:%S")
fDate          = datetime_RUN.strftime("%Y%m%d")
runTime        = "t" + datetime_RUN.strftime("%H") + "z"

# Path Setup
download_path_CK    = download_path + '/' + product + '/' + fDate + '/' + runTime + '/' + 'out_' + product + '_chunked.nc'
download_path_NC_A  = download_path + '/' + product + '_tmp.nc'

if not os.path.exists(download_path + '/' + product):
    os.makedirs(download_path + '/' + product)
if not os.path.exists(download_path + '/' + product + '/' + fDate):
    os.makedirs(download_path + '/' + product + '/' + fDate)
if not os.path.exists(download_path + '/' + product + '/' + fDate + '/' + runTime):
    os.makedirs(download_path + '/' + product + '/' + fDate + '/' + runTime)

In [ ]:
# Setup grid transformation parameters 
HRRR_grid1 = 'lambert:262.500000:38.500000:38.500000:38.500000'
HRRR_grid2 = '237.280472:1799:3000.000000'
HRRR_grid3 = '21.138123:1059:3000.000000'


In [ ]:
# Download Grib File from AWS Open Data
# Setup file names
grbType   = 'wrfsfc'

# Set the range of forecasts to download
ncFileRange_A = range(1, 7)

for ncFileName in ncFileRange_A:
    download_file_pathA  = download_path + '/hrrrh.f' + str(ncFileName).zfill(3) + '.' + grbType + '.grb'
    download_path_GB_A   = download_path + '/hrrrh.f' + str(ncFileName).zfill(3) + '.' + grbType + '.grb.earth'

    s3_filename = 'hrrr.' + fDate + '/conus/hrrr.' + runTime + '.wrfsfcf' + str(ncFileName).zfill(2) + '.grib2'

    # Download the grib file
    s3_client.download_file(bucket, s3_filename, download_file_pathA)

    #%% Process Downloaded File

    # Set variables to read
    matchString = (":(TMP:2 m above ground|CRAIN:surface|CSNOW:surface|"
                   "CFRZR:surface|PRATE:surface|PRES:surface|CICEP:surface|"
                   "UGRD:10 m above ground:.*hour fcst|"
                   "VGRD:10 m above ground:.*hour fcst|"
                   "VIS:surface|DPT:2 m above ground|TCDC:entire atmosphere|GUST:surface|RH:2 m above ground):")

    # Extract variables and convert winds from grid relative to earth relative
    pywgrib2_s.wgrib2([download_file_pathA, '-new_grid_winds', 'earth', '-new_grid_interpolation', 'neighbor', '-match', matchString, '-new_grid', HRRR_grid1, HRRR_grid2, HRRR_grid3, download_path_GB_A])
    # Add precipitation
    pywgrib2_s.wgrib2([download_file_pathA, '-rewind_init', download_file_pathA, '-new_grid_winds', 'earth', '-new_grid_interpolation', 'neighbor', '-match', 'APCP', '-append','-new_grid', HRRR_grid1, HRRR_grid2, HRRR_grid3, download_path_GB_A, '-quit'])
    pywgrib2_s.close(download_path_GB_A)

    # Add to NetCDF
    pywgrib2_s.wgrib2([download_path_GB_A, '-append', '-netcdf', download_path_NC_A])

    pywgrib2_s.close(download_file_pathA)
    pywgrib2_s.close(download_path_GB_A)

    os.remove(download_file_pathA)
    os.remove(download_path_GB_A)

    print(ncFileName)

In [ ]:
# Chunk NetCDF
# Remove Chunk file if exists
if os.path.isfile(download_path_CK):
  os.remove(download_path_CK)
if os.path.isfile(download_path  + '/' + product + '/' + fDate + '/' + runTime + '/' + product + '.done'):
        os.remove(download_path  + '/' + product + '/' + fDate + '/' + runTime + '/' + product + '.done')


In [ ]:
# Setup New Datasets
chkA    = Dataset(download_path_CK, "w")
srcA    = Dataset(download_path_NC_A, 'r', format="NETCDF3_CLASSIC")

# Copy global attributes all at once via dictionary
src = srcA
chk = chkA
chk.setncatts(srcA.__dict__)



In [ ]:
#%% Copy dimensions for source NetCDF
for name, dimension in src.dimensions.items():
    chk.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))


In [ ]:
#%% Copy all file data
# This step puts the NetCDF file into time oriented chuncks, as well as adding compression
for name, variable in src.variables.items():
    if len(variable.dimensions)==3:
        if 'PRATE_surface' in name:
            x = chk.createVariable(name, variable.datatype, variable.dimensions, chunksizes=[30, 10, 10], zlib=True, least_significant_digit=4, complevel=1)
        else:
            x = chk.createVariable(name, variable.datatype, variable.dimensions, chunksizes=[30, 10, 10], zlib=True, least_significant_digit=1, complevel=1)
    else:
        x = chk.createVariable(name, variable.datatype, variable.dimensions)

    chk[name][:] = src[name][:]
    # copy variable attributes all at once via dictionary
    for ncattr in src[name].ncattrs():
        if ncattr != '_FillValue':
            chk[name].setncattr(ncattr, src[name].getncattr(ncattr))

src.close()
chk.close()



In [ ]:
ChunkNCfile = Dataset(download_path_CK, 'r')

# Read Latitude, Longitude, and Times points in file
lats_hrrr = ChunkNCfile['latitude'][:, :]
lons_hrrr = ChunkNCfile['longitude'][:, :]
times_hrrr = ChunkNCfile['time'][:]

# Convert timestamps to datetimes
datetime_hrrr = []
for i in range(0, len(times_hrrr)):
    datetime_hrrr.append(datetime.fromtimestamp(times_hrrr[i]))

# Set desired lat long
##### NOTE #####
# Must be in the continental United States or southern Canada
Point_Lat = 45.4215
Point_Lon = -75.6972+360

# Find cloest point
abslat = np.abs(lats_hrrr-Point_Lat)
abslon = np.abs(lons_hrrr-Point_Lon)
c = np.maximum(abslon, abslat)
x_hrrr, y_hrrr = np.where(c == np.min(c))

# Exract and print temperature forecast!
vard = ChunkNCfile['TMP_2maboveground'][:, x_hrrr, y_hrrr]

# Convert to degrees Celsius
print(vard-272.15)


In [ ]:
fig = plt.figure()

plt.plot(datetime_hrrr, vard[:, 0, 0]-272.15)
plt.xlabel('Datetime')
plt.ylabel('Temperature [degC]')
plt.show()
